In [ ]:
import requests
import tkinter as tk
from tkinter import messagebox
import json
from datetime import datetime

# Constants
API_KEY = "your_openweathermap_api_key"  # Replace with your OpenWeatherMap API key
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
FORECAST_URL = "http://api.openweathermap.org/data/2.5/forecast"

# Fetch current weather data
def fetch_weather(city):
    """
    Fetch weather data for a given city using OpenWeatherMap API.

    Args:
        city (str): Name of the city.

    Returns:
        dict: Weather details including temperature, humidity, wind speed, conditions, and local time.
    """
    try:
        params = {
            'q': city,
            'appid': API_KEY,
            'units': 'metric'
        }
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        if data.get("cod") != 200:
            messagebox.showerror("Error", data.get("message", "Failed to fetch weather data"))
            return None

        local_time = datetime.utcfromtimestamp(data["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        return {
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"],
            "conditions": data["weather"][0]["description"],
            "local_time": local_time
        }

    except Exception as e:
        messagebox.showerror("Error", str(e))
        return None

# Fetch 5-day weather forecast
def fetch_forecast(city):
    """
    Fetch 5-day weather forecast for a given city.

    Args:
        city (str): Name of the city.

    Returns:
        list: List of forecast details for the next 5 days.
    """
    try:
        params = {
            'q': city,
            'appid': API_KEY,
            'units': 'metric'
        }
        response = requests.get(FORECAST_URL, params=params)
        data = response.json()

        if data.get("cod") != "200":
            messagebox.showerror("Error", data.get("message", "Failed to fetch forecast data"))
            return None

        forecast = []
        for entry in data["list"]:
            time = datetime.utcfromtimestamp(entry["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            forecast.append({
                "time": time,
                "temperature": entry["main"]["temp"],
                "conditions": entry["weather"][0]["description"]
            })

        return forecast

    except Exception as e:
        messagebox.showerror("Error", str(e))
        return None

# Save favorite locations
def save_favorite_location(city):
    """
    Save a city to the list of favorite locations in a JSON file.

    Args:
        city (str): Name of the city.
    """
    try:
        with open("favorites.json", "r") as file:
            favorites = json.load(file)
    except FileNotFoundError:
        favorites = []

    if city not in favorites:
        favorites.append(city)

    with open("favorites.json", "w") as file:
        json.dump(favorites, file)

    messagebox.showinfo("Success", f"{city} added to favorites")

# Load favorite locations
def load_favorite_locations():
    """
    Load the list of favorite locations from a JSON file.

    Returns:
        list: List of favorite city names.
    """
    try:
        with open("favorites.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return []

# Remove a favorite location
def remove_favorite_location(city):
    """
    Remove a city from the list of favorite locations.

    Args:
        city (str): Name of the city.
    """
    try:
        with open("favorites.json", "r") as file:
            favorites = json.load(file)

        if city in favorites:
            favorites.remove(city)
            with open("favorites.json", "w") as file:
                json.dump(favorites, file)
            messagebox.showinfo("Success", f"{city} removed from favorites")
        else:
            messagebox.showinfo("Error", f"{city} is not in favorites")
    except FileNotFoundError:
        messagebox.showinfo("Error", "No favorite locations found")

# GUI Setup
def weather_app():
    """
    Launch the Weather Forecast Application GUI.
    """
    def search_weather():
        city = city_entry.get()
        weather_data = fetch_weather(city)
        if weather_data:
            result_label.config(
                text=(
                    f"Temperature: {weather_data['temperature']} °C\n"
                    f"Humidity: {weather_data['humidity']}%\n"
                    f"Wind Speed: {weather_data['wind_speed']} m/s\n"
                    f"Conditions: {weather_data['conditions']}\n"
                    f"Local Time: {weather_data['local_time']}"
                )
            )

    def display_forecast():
        city = city_entry.get()
        forecast_data = fetch_forecast(city)
        if forecast_data:
            forecast_text = "\n".join(
                f"{item['time']}: {item['temperature']} °C, {item['conditions']}"
                for item in forecast_data
            )
            result_label.config(text=f"5-Day Forecast:\n{forecast_text}")

    def add_to_favorites():
        city = city_entry.get()
        if city:
            save_favorite_location(city)

    def show_favorites():
        favorites = load_favorite_locations()
        if favorites:
            messagebox.showinfo("Favorites", "\n".join(favorites))
        else:
            messagebox.showinfo("Favorites", "No favorite locations saved.")

    def remove_favorite():
        city = city_entry.get()
        if city:
            remove_favorite_location(city)

    # Main window
    window = tk.Tk()
    window.title("Weather Forecast App")
    window.configure(bg="gray")

    tk.Label(window, text="Enter City:", bg="gray", fg="white").pack(pady=5)
    city_entry = tk.Entry(window)
    city_entry.pack(pady=5)

    tk.Button(window, text="Search Weather", command=search_weather).pack(pady=5)
    tk.Button(window, text="5-Day Forecast", command=display_forecast).pack(pady=5)
    tk.Button(window, text="Add to Favorites", command=add_to_favorites).pack(pady=5)
    tk.Button(window, text="Show Favorites", command=show_favorites).pack(pady=5)
    tk.Button(window, text="Remove from Favorites", command=remove_favorite).pack(pady=5)

    result_label = tk.Label(window, text="", justify=tk.LEFT, bg="gray", fg="white")
    result_label.pack(pady=10)

    window.mainloop()

if __name__ == "__main__":
    weather_app()
